In [1]:
import pandas as pd
import numpy as np
import csv
from neo4j import GraphDatabase
from neo4j.debug import watch
import configparser

In [2]:
df = pd.read_csv('./groups_techniques.csv',header=0,index_col=0)
G_all=df.index
T_all=list(df.columns)

In [3]:
df

,T1548,T1134,T1531,T1087,T1098,T1583,T1595,T1557,T1071,T1010,...,T1535,T1550,T1204,T1078,T1125,T1497,T1600,T1102,T1047,T1220
G0018,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
G0130,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
G1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G0138,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
G0099,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G0112,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
G0044,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G0090,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
G0102,0,0,0,1,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0


In [4]:
matrix=[]
with open('./matrix.csv') as f:
    for row in csv.reader(f):
        matrix.append(row)
matrix.pop(0)
for i in range(len(matrix)):
    matrix[i].remove(str(i))
    matrix[i].remove(matrix[i][0])
for i in range(len(matrix)):
    while '' in matrix[i]:
        matrix[i].remove('')

In [6]:
TA_all=[]
with open('./matrix.csv') as f:
    for row in csv.reader(f):
        TA_all.append(row[1])
TA_all.pop(0)

'0'

In [9]:
def get_e_stage(technique):
    for i in range(len(matrix)): #len(matrix) is 14
        if technique in matrix[i]:
            return i

In [10]:
def get_l_stage(technique):
    for i in reversed(range(len(matrix))):#len(matrix) is 14
        if technique in matrix[i]:
            return i

In [11]:
#データベースをクリアする
def clear_db(tx):
    tx.run('MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r')

#Nodeを追加する
def add_group_node(tx):
    tx.run('LOAD CSV WITH HEADERS FROM "file:///Groups.csv" AS row CREATE (:Group {name: row.name, similarity:0.0});')
    
def add_technique_node(tx):
    tx.run('LOAD CSV WITH HEADERS FROM "file:///Techniques.csv" AS row CREATE (:Technique {name: row.name, value: row.value,recommend: 0, input:0, sr:0});')
    
def add_tactic_node(tx):
    tx.run('LOAD CSV WITH HEADERS FROM "file:///Tactics.csv" AS row CREATE (:Tactic {name: row.name, phase: row.stage});')

def add_adversary_node(tx, name):
    tx.run('CREATE (g:Group {name: $name}) RETURN g', {'name': name})
    
#Nodeにプロパティを追加する
def add_node_property_e_stage(tx,name,e_stage):
    tx.run('MATCH (t:Technique {name:$name}) SET t.e_stage = $e_stage RETURN t',name=name, e_stage=e_stage)
    
def add_node_property_l_stage(tx,name,l_stage):
    tx.run('MATCH (t:Technique {name:$name}) SET t.l_stage = $l_stage RETURN t',name=name, l_stage=l_stage)
    
#エッジを追加する
def add_used_relationship(tx, group, technique):
    tx.run('MATCH (g:Group {name: $group}) '
           'MATCH (t:Technique {name: $technique}) '
           'CREATE (g)-[:USED {weight:0.0}]->(t)',
            group=group, technique=technique)

def add_next_relationship(tx, pre_tactic, nxt_tactic):
    tx.run('MATCH (pre_ta:Tactic {name: $pre_tactic}) '
           'MATCH (nxt_ta:Tactic {name: $nxt_tactic}) '
           'CREATE (pre_ta)-[:NEXT]->(nxt_ta)',
            pre_tactic=pre_tactic, nxt_tactic=nxt_tactic)

def add_contains_relationship(tx, tactic, technique):
    tx.run('MATCH (ta:Tactic {name: $tactic}) '
           'MATCH (t:Technique {name: $technique}) '
           'CREATE (ta)-[:CONTAINS]->(t)',
            tactic=tactic, technique=technique)

In [12]:
def make_graph_databases():
    #neo4jの設定取得
    config = configparser.ConfigParser()
    config.read('neo4j.ini')
    uri = config['NEO4J']['uri']
    user = config['NEO4J']['user']
    password = config['NEO4J']['password']
    
    # neo4jドライバーの作成
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    #グラフデータベースの作成開始
    with driver.session() as session:
        session.write_transaction(clear_db)
        session.write_transaction(add_group_node)
        session.write_transaction(add_adversary_node,"Adversary")
        session.write_transaction(add_technique_node)
        session.write_transaction(add_tactic_node)
        
        for t in T_all:
            e_stage=get_e_stage(t)
            session.write_transaction(add_node_property_e_stage, t, e_stage)
            l_stage=get_l_stage(t)
            session.write_transaction(add_node_property_l_stage, t, l_stage)
        
        
        for group in G_all:
            i=df.index.get_loc(group)
            for j in range(len(T_all)):
                if df.iloc[i][j]==1:
                    session.write_transaction(add_used_relationship, group, T_all[j])
    
        for i in range(len(TA_all)):
            for technique in matrix[i]:
                session.write_transaction(add_contains_relationship,TA_all[i],technique)
        
        for i in range(len(TA_all)-1):
            session.write_transaction(add_next_relationship,TA_all[i],TA_all[i+1])
        

In [13]:
make_graph_databases()